### Import the libraries

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import os
import sys
from pathlib import Path

import plotly
import plotly.graph_objects as go

import matplotlib.pyplot as plt

#import fbprophet 
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly, plot_yearly, plot_weekly, plot_cross_validation_metric
from fbprophet.diagnostics import cross_validation, performance_metrics

import logging
logging.getLogger('fbprophet').setLevel(logging.CRITICAL)

In [2]:
notebook_path = Path(os.getcwd())
root_path = notebook_path.parent.absolute()
os.chdir(root_path)
str(root_path)

'/Users/Antoine/data_science_projects/natixis_challenge'

In [3]:
from src.model import evaluate, forecast
from src.seasonality import find_closest, clean, clean_multiple, format_output
import datetime
from datetime import timedelta

### Turn off fbprophet logs and FutureWarnings

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# from https://stackoverflow.com/questions/11130156/suppress-stdout-stderr-print-from-python-functions
class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

### Load the datasets

In [6]:
item_trend = pd.read_csv('./data/item_trend_20221221.csv', index_col=0)
df_item_info = pd.read_csv('./data/item_info_20221221.csv', index_col=0)
df_tmp_hosts_zabbix = pd.read_csv('./data/tmp_hosts_zabbix_20221221.csv', index_col=0)
df_cockpit = pd.read_csv('./data/cockpit_20221221.csv', index_col=0)
df_mycloud = pd.read_csv('./data/mycloud_20221221.csv', index_col=0)
item_periods = pd.read_csv('./data/itemid_and_periods.csv', index_col=0)

In [7]:
print(item_periods.shape)
item_periods.head(2)

(6266, 4)


,itemid,period1,period2,period3
0,2206,7.0,NaN,NaN
1,2269,NaN,NaN,NaN


In [8]:
print(df_tmp_hosts_zabbix.shape)
df_tmp_hosts_zabbix.head(2)

(4365, 2)


,hostid,host
0,6,slpdfrora040a
1,800,SWPAFRWNETV44


In [9]:
print(df_cockpit.shape)
df_cockpit.head(2)

(5189, 22)


,name_department,iua,name_subfunction,name_function,name_server,model,name_state,name_environment,os,country,...,ram,number_core,number_cpu,type_cpu,electrical_power,server_parent,server_parent2,name_cluster,mycloud,bcloud
0,etu-bgc-etrading-architecture,AV9,Application Server,APPLICATION,SWDCFRAV9779,VIRTUAL_MACHINE,INFUNCTION,Dev,Windows 2016 Standard 10.0.14393,FRANCE,...,16777,2,2,INTEL(R) XEON(R) SILVER 4314 CPU @ 2.40GHZ,NaN,SEPCFRNXFX10808,NaN,NaN,Yes,NaN
1,etu-bgc-etrading-architecture,Y59,Application Server,APPLICATION,SLPAFRETR168,VIRTUAL_MACHINE,INFUNCTION,Prod,Linux 3.10.0-1160.76.1.el7.x86_64 RHEL7.9,FRANCE,...,7629,1,1,INTEL(R) XEON(R) CPU E5-2680 0 @ 2.70GHZ:X86_64,NaN,SEPIFRLINP029,NaN,NaN,No,NaN


In [10]:
print(item_trend.shape)
item_trend.head(2)

(3866173, 6)


,itemid,clock,value_min,value_avg,value_max,item_type
0,264670,2021-05-27,5.789395,7.956363,50.884403,cpu
1,264670,2021-05-28,5.737689,10.279124,63.836303,cpu


In [11]:
print(df_item_info.shape)
df_item_info.head(2)

(10015, 3)


,itemid,hostid,item_type
0,264670,806,cpu
1,622332,4755,cpu


In [12]:
print(item_trend.shape)
item_trend.head(2)

(3866173, 6)


,itemid,clock,value_min,value_avg,value_max,item_type
0,264670,2021-05-27,5.789395,7.956363,50.884403,cpu
1,264670,2021-05-28,5.737689,10.279124,63.836303,cpu


### Select a time-series

In [13]:
item_trend.itemid.unique()[:5]

array([264670, 622332, 629146, 664016, 664732])

In [14]:
# Convert date strings to date 
item_trend["clock"] = pd.to_datetime(item_trend.clock).copy()

sample = item_trend[item_trend.itemid==664016]

# Take the max value 
df = pd.DataFrame({'ds': sample.clock, "y": sample.value_max})

print(df.shape)
df.head(2)

(573, 2)


,ds,y
1719,2021-05-27,60.993978
1720,2021-05-28,99.491720


In [15]:
date_min, date_max = df.ds.min(), df.ds.max()
print(f"Date range : from {str(date_min)} to {str(date_max)}.")

Date range : from 2021-05-27 00:00:00 to 2022-12-20 00:00:00.


### Train/test split

In [17]:
#split_date = df.ds.min().replace(year = df.ds.min().year+1)
#print(split_date)
#train, valid = df.loc[df['ds'] < split_date].copy(), df.loc[df['ds'] >= split_date].copy()

# only keep last year, take 100% for the training (no hyperparameter tuning)
train = df.tail(365).copy()
valid = pd.DataFrame({'ds':[], 'y':[]})


fig = go.Figure()
fig.add_trace(go.Scatter(x=train["ds"], y=train["y"], mode="markers", name='Training', line=dict(color='#002244')))
fig.add_trace(go.Scatter(x=valid["ds"], y=valid["y"], mode="markers", name='Validation', line=dict(color='#ff0066')))
fig.update_layout(yaxis_title='CPU usage', width=800, height=400)
fig.show()

### Model

In [19]:
# Set parameters
params = dict()

# Saturation at 100
params['growth'] = 'logistic'
train["cap"] = 100
valid["cap"] = 100

# Weight of each seasonlity
params['seasonality_prior_scale'] = 10

# Uncertainty intervals
#params["mcmc_samples"] = 300 # too long to compute
params["interval_width"] = 0.95

In [21]:
# Create the model 
model = Prophet(**params)
model.add_seasonality(period=30.5, name='monthly', fourier_order=5)
model.add_seasonality(period=7, name='weekly', fourier_order=3)

# Fit and predict
with suppress_stdout_stderr():
    pred = forecast(model, train, valid)
pred.head(2)

,ds,trend,cap,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,...,monthly_lower,monthly_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y
0,2021-12-21,50.330128,100,45.673994,74.366333,50.330128,50.330128,10.742893,10.742893,10.742893,...,0.661579,0.661579,10.081315,10.081315,10.081315,0.0,0.0,0.0,61.073021,54.895771
1,2021-12-22,50.307892,100,46.043745,74.514275,50.307892,50.307892,10.103123,10.103123,10.103123,...,0.133024,0.133024,9.970099,9.970099,9.970099,0.0,0.0,0.0,60.411014,55.764326


In [22]:
# Evaluate model
to_evaluate = pd.DataFrame({'y': train.set_index(train.ds)['y'], 'yhat': pred.set_index(pred.ds)["yhat"]})
evaluate(to_evaluate, valid)

{'RMSE': 7.256858220371186, 'MAPE': 0.13251521286191947}

### Decompose seasonalities

In [24]:
with suppress_stdout_stderr():
    component_plot = plot_components_plotly(model, pred)
component_plot

### Final reconstruction

In [25]:
# Plot final reconstruction
fig = plot_plotly(model, pred, changepoints=True, trend=True)
fig.update_layout(title_text=f'Logistic function', title_x=0.5)
fig.show()

### Global pipeline

In [31]:
def clean(train, valid, seasonality_periods=[7, 30.5]):
    '''Modelise a time series through its seasonalities only : clean out ponctual events.

    Args:
        train : pd.DataFrame = training set time-series
        valid : pd.DataFrame = validation set time-series
        list_seasonalities : list(int) = list of the periods of each seasonality
    
    Output:
        new_df : pd.DataFrame = cleaned time series
    '''

    ######################
    #PARAMETERS
    params = dict()
    # Saturation at 100
    params['growth'] = 'logistic'
    train["cap"] = 100
    valid["cap"] = 100

    # Weight of each seasonlity
    params['seasonality_prior_scale'] = 10

    # Uncertainty intervals
    params["interval_width"] = 0.95

    # Remove seasonalities to add them ourselves
    #params["yearly_seasonality"] = False
    #params["monthly_seasonality"] = False
    params["weekly_seasonality"] = False
    #######################

    # Set model
    model = Prophet(**params)
    for period in seasonality_periods:
        model.add_seasonality(period=period, name=f'seasonality_{period}', fourier_order=5)

    # Predict
    df = pd.concat([train, valid], axis=0)
    with suppress_stdout_stderr():
        model.fit(train)
    pred = model.predict(df)
    pred['y'] = np.array(df.y)

    # Evaluate
    to_evaluate = pd.DataFrame({'y': df.set_index(df.ds)['y'], 'yhat': pred.set_index(pred.ds)["yhat"]})
    scores = evaluate(to_evaluate, valid)

    # Plot seasonal decomposition
    #season_fig = plot_components_plotly(model, pred)

    # Plot final time series
    #ts_fig = plot_plotly(model, pred, changepoints=True, trend=True)
    #ts_fig.update_layout(title_text=f'Logistic function', title_x=0.5)

    #print(f"Scores : {scores}.")
    return pred[["ds", "y", "yhat_lower", "yhat", "yhat_upper"]]#, season_fig, ts_fig

In [27]:
output = clean(train, valid)
print(output.shape)
output.head()

Scores : {'RMSE': 7.2591891772703345, 'MAPE': 0.13214577640995792}.
(365, 5)


,ds,y,yhat_lower,yhat,yhat_upper
0,2021-12-21,54.895771,46.984034,61.152817,74.865452
1,2021-12-22,55.764326,46.334234,60.484823,75.204762
2,2021-12-23,55.091282,47.115004,60.994587,74.857157
3,2021-12-24,15.647099,42.642874,57.213691,71.711889
4,2021-12-25,18.091388,9.257728,24.053189,38.160684


### Process multiple time-series

In [28]:
# Sample of the dataframe indicating the periods of each time-series.
df_arsene_sample = pd.DataFrame({'itemid': [264670, 622332, 629146],\
     'Period 1': [30.5, 30.5, 30.5], 
     'Period 2':[7, 7, 7], 
     'Period 3': [np.nan, np.nan, np.nan]})
df_arsene_sample

,itemid,Period 1,Period 2,Period 3
0,264670,30.5,7,NaN
1,622332,30.5,7,NaN
2,629146,30.5,7,NaN


In [68]:
def clean_multiple(item_trend, df_periods, duration=365):
    '''
    Apply the seasonality cleaning on multiple time-series along their seasonality periods. 

    Args:
        item_trend : pd.DataFrame = table of the time series, with their ids
        df_periods : pd.DataFrame = table listing the seasonality periods of each time-series
        duration : int = number of last days to take into account in the time-series
    
    Output:
        final_df : pd.DataFrame = table of the cleaned time-series
    '''
    final_df = pd.DataFrame(columns=["ds", "y", "yhat", "yhat_upper", "itemid"])
    for item_idx in tqdm(range(len(df_periods))):
        item = df_periods.loc[item_idx, :]
        item_id = item.itemid
        periods = item.drop('itemid')
        periods = [period for period in periods if str(period)!="nan"]

        # Original time-series
        sample = item_trend[item_trend.itemid==item_id].sort_values('clock', ascending=True).tail(duration)
        df = pd.DataFrame({'ds': sample.clock, "y": sample.value_max}) # We keep max value for dimensioning
        df['ds'] = pd.to_datetime(df['ds'])

        # Train-test split : we use 100% for training (no hyperparameter tuning)
        train = df
        valid = pd.DataFrame({'ds':[], 'y':[]})

        # If there are periods, we clean the time-serie
        if len(periods)!=0:
            # Clean time series
            output = clean(train, valid, periods)
        else:
            output = df.copy()
        output["itemid"] = item_id

        # Collect output
        final_df = pd.concat([final_df, output], axis=0)
    
    return final_df

In [32]:
final_df = clean_multiple(item_trend, df_arsene_sample)
print(final_df.shape)
final_df.head(2)

100%|██████████| 3/3 [00:10<00:00,  3.60s/it]

(1095, 6)


,ds,y,yhat,yhat_upper,itemid,yhat_lower
0,2021-12-21,65.765696,82.137661,104.652478,264670.0,60.041047
1,2021-12-22,99.055944,84.137089,105.055490,264670.0,61.519349


In [33]:
print(f"Data existing for all days forgotten : {len(final_df) == 365*df_arsene_sample.itemid.nunique()}.")

Data existing for all days forgotten : True.


### Adapt format for Lucas

- une colonne item_type ou à chaque ligne tu as “cpu” ou “mem”
- 2 colonnes “number_cpu” et “ram”,  infos serveur même si la table est au niveau de l’item_id (genre la ram d’un serveur doit apparaitre dans 2 lignes, celle de l’item_id ram et celle de item_id CPU de ce serveur)
- une colonne usage qui correspond à notre colonne qui doit rester entre 2 seuils (donc tu peux choisir val_avg etc)

In [34]:
def format_output(ts, df_item_info, df_tmp_hosts_zabbix, df_cockpit, df_mycloud):
    '''
    Format output dataframe for the final capacity optimization.

    Args:
        output : pd.DataFrame = table of the time-series
        df_item_info : pd.DataFrame([item type, server id]) = table of the item information 
        df_tmp_hosts_zabbix : pd.DataFrame([server name, server id]) = table referencing the name of each server 
        df_cockpit : pd.DataFrame = table referencing the server used for each application
        df_mycloud : pd.DataFrame = catalogue table of the servers
    Output:
        formatted_output : pd.DataFrame = formatted table
    '''
    formatted_output = ts.copy()\
        .merge(df_item_info, on='itemid', how='left')\
        .merge(df_tmp_hosts_zabbix, on='hostid', how='left')\
        .merge(df_cockpit[["name_server", "ram", "number_cpu"]], right_on='name_server', left_on='host')\
        .drop(["hostid", "host", "y"], axis=1)
    
    formatted_output["itemid"] = formatted_output["itemid"].astype('int')

    # Find actual server ram
    formatted_output["ram"] = formatted_output.ram/1000
    formatted_output["ram_server"] = formatted_output.ram.apply(lambda x: find_closest(x, df_mycloud.RAM.unique()))
    formatted_output = formatted_output.drop(["ram"], axis=1)

    return formatted_output

In [35]:
final_df_formatted = format_output(final_df, df_item_info, df_tmp_hosts_zabbix, df_cockpit, df_mycloud)

print("Number of servers per server type:")
final_df_formatted[['name_server', 'number_cpu', 'ram_server']]\
    .rename(columns={"name_server" : "Number of servers"})\
    .groupby(['number_cpu', 'ram_server'])\
    .nunique()

Number of servers per server type:


,,Number of servers
number_cpu,ram_server,
1,4,1
2,4,1
4,12,1


### Apply to all the time-series having seasonalities

In [66]:
# Remove the items having no seasonality
cleaned_item_periods = item_periods[item_periods.period1.apply(lambda x: ~np.isnan(x))].reset_index(drop=True).copy()
print(f"There are {len(cleaned_item_periods)} seasonal items ({round(100*len(cleaned_item_periods)/len(item_periods))}%).")
cleaned_item_periods.head(2)

There are 771 seasonal items (12%).


,itemid,period1,period2,period3
0,2206,7.0,NaN,NaN
1,606770,7.0,NaN,NaN


In [62]:
np.unique(cleaned_item_periods[["period1", "period2", "period3"]].values)

array([ 3.,  5.,  6.,  7., 10., 14., 21., 28., 35., nan])

In [69]:
result1 = clean_multiple(item_trend, cleaned_item_periods)
result1.head(2)

 47%|████▋     | 363/771 [21:27<20:43,  3.05s/it]  /Users/Antoine/data_science_projects/natixis_challenge/src/model.py:21: RuntimeWarning:

divide by zero encountered in true_divide

 62%|██████▏   | 481/771 [27:27<15:16,  3.16s/it]/Users/Antoine/data_science_projects/natixis_challenge/src/model.py:21: RuntimeWarning:

divide by zero encountered in true_divide

100%|██████████| 771/771 [41:39<00:00,  3.24s/it]


,ds,y,yhat,yhat_upper,itemid,yhat_lower
0,2021-12-20,92.497442,76.348421,112.217324,2206.0,42.565523
1,2021-12-21,88.463228,78.672104,114.191701,2206.0,46.499314


In [70]:
result2 = format_output(result1, df_item_info, df_tmp_hosts_zabbix, df_cockpit, df_mycloud)
print(result2.shape)
result2.head(2)

(249295, 9)


,ds,yhat,yhat_upper,itemid,yhat_lower,item_type,name_server,number_cpu,ram_server
0,2021-12-21,91.586303,103.343355,606770,80.166515,mem_pused,SWPAFRETRADM2,1,4
1,2021-12-22,92.441946,103.402919,606770,80.972322,mem_pused,SWPAFRETRADM2,1,4


In [71]:
result2.to_csv("data/item_ts_seasonalized.csv", index=False)